In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate
from scipy.optimize import approx_fprime
from mpl_toolkits.mplot3d import Axes3D

# import numdifftools as nd
# np.tensordot
# np.cross
# np.dot

In [43]:
def vt(X: np.ndarray) -> np.ndarray:
    """Returns the tangent and azimutal unit vectors at point X on the unit sphere.
    Args:
        X (np.ndarray): A 3D point on the unit sphere.
        Returns:
        [-y/(x^2+y^2)^0.5, x/(x^2+y^2)^0.5, 0]"""
    s = np.sqrt(X[0]**2 + X[1]**2)
    return np.array([-X[1]/s, X[0]/s, 0])

def vr(X: np.ndarray) -> np.ndarray:
    """Returns the radial unit vector at point X on the unit sphere.
    Args:
        X (np.ndarray): A 3D point on the unit sphere.
        Returns:
        [X[0], X[1], X[2]]/(np.sqrt(X[0]**2 + X[1]**2 + X[2]**2))"""
    s = np.sqrt(X[0]**2 + X[1]**2+ X[2]**2)
    return np.array([X[0], X[1], X[2]])/s

def th(X: np.ndarray):
    """Returns the polar angle theta at point X on the unit sphere theta."""
    return np.arccos(X[2]/np.sqrt(X[0]**2 + X[1]**2 + X[2]**2))

def ncone(X: np.ndarray, w) -> np.ndarray:
    """Returns the director n on the boundary of the cone at point X on the unit sphere.
    Args:
        X (np.ndarray): A 3D point on the unit sphere.
        thetacone (float): The half-angle of the cone in radians.
        w (int): The winding number of the director along the boundary on the cone.
        Returns:
        n = cos(w*phi)*vt + sin(w*phi)*vr
        where phi = arctan2(y,x)
        
        if x=0 then we separate fo different cases"""
    if X[0] == 0 and X[1] == 0:
        return np.zeros(3)
    elif X[0] == 0 and X[1] > 0:   
        return np.cos(w * np.pi / 2) * vt(X) + np.sin(w * np.pi / 2) * vr(X)
    elif X[0] == 0 and X[1] < 0:
        return np.cos(w * np.pi / 2) * vt(X) - np.sin(w * np.pi / 2) * vr(X)        
    else:
        return np.cos(w * np.arctan2(X[1], X[0])) * vt(X) + np.sin(w * np.arctan2(X[1], X[0])) * vr(X)
    


def nhat(X, thetacone: float, w) -> np.ndarray:
    """Returns the director n at point X on the unit sphere.
    Args:
        X (np.ndarray): A 3D point on the unit sphere.
        thetacone (float): The half-angle of the cone in radians.
        w (int): The winding number of the director along the boundary on the cone.
        Returns:
        n = """
    n = th(X) * ncone(X, w) + (thetacone - th(X)) * np.array([0, 0, 1])
    return n / np.linalg.norm(n)


In [9]:
def epsilon3d():
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1
    return eps

In [47]:
def integrand(point, angle, winding) -> np.ndarray:
    """epsilon_ijk epsilon_pqr n_q,i n_r,k n_p at point on the unit sphere.
    Args:
        point (np.ndarray): A 3D point on the unit sphere.
        angle (float): The half-angle of the cone in radians.
        winding (int): The winding number of the director along the boundary on the cone.
        Returns:
        A 3D vector representing the integrand at the given point."""
    grad_nx = approx_fprime(point, lambda p: nhat(p, angle, winding)[0], epsilon=1e-8)  # x-component - 3d vector
    grad_ny = approx_fprime(point, lambda p: nhat(p, angle, winding)[1], epsilon=1e-8)  # y-component - 3d vector
    grad_nz = approx_fprime(point, lambda p: nhat(p, angle, winding)[2], epsilon=1e-8)  # z-component - 3d vector

    n = nhat(point, angle, winding)  # 3d vector
    dn = np.stack((grad_nx, grad_ny, grad_nz), axis=0)  # 3x3 matrix
    r = np.array([point[0], point[1], point[2]])

    return np.round(np.einsum('ijk,pqr,qj,rk,p,i', epsilon3d(), epsilon3d(), dn, dn, n, r) * th(point), decimals=8) # 3d vector


In [48]:
integrand(np.array([0,1,0]), np.pi/4, 1)

2e-08

In [ ]:
result, error = integrate.dblquad(integrand_with_jacobian, 
                                  phi_min, phi_max, 
                                  theta_min_func, theta_max_func)